In [1]:
# Чтобы скачать и распаковать zip-архив в output папку в Питоне:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go 

from IPython.display import display

#!/usr/bin/env python
import io
import zipfile
import requests  # $ pip install requests

pd.set_option('display.max_columns', 40) # выводить больше столбцов
pd.set_option('display.max_rows', 40)   # выводить больше строк
plt.style.use('default')    # корректный вывод графиков в темной теме 
# fig.show(renderer='notebook') #включение анимации

r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/e157dad19d7511561f697586ed146dfe/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/churn.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('Data')

# https://lms.skillfactory.ru/assets/courseware/v1/e157dad19d7511561f697586ed146dfe/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/churn.zip

In [2]:
churn_data = pd.read_csv('data/churn.csv')
churn_data = churn_data.drop(labels='RowNumber', axis=1)
display(churn_data.head())
# move to category type
categories = ['IsActiveMember', 'HasCrCard', 'Gender', 'Geography']
for column in categories:
    churn_data[column] = churn_data[column].astype('category')
    
# Exited status change to loyle & lost clients
churn_data['Status'] = churn_data['Exited'].apply(lambda x: 'Лояльные клиенты' if x == 0 else 'Ушедшие клиенты')
# churn_data = churn_data.drop('Exited', axis=1)
display(churn_data.head())

churn_data.info()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Status
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,Ушедшие клиенты
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,Лояльные клиенты
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,Ушедшие клиенты
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,Лояльные клиенты
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,Лояльные клиенты


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CustomerId       10000 non-null  int64   
 1   Surname          10000 non-null  object  
 2   CreditScore      10000 non-null  int64   
 3   Geography        10000 non-null  category
 4   Gender           10000 non-null  category
 5   Age              10000 non-null  int64   
 6   Tenure           10000 non-null  int64   
 7   Balance          10000 non-null  float64 
 8   NumOfProducts    10000 non-null  int64   
 9   HasCrCard        10000 non-null  category
 10  IsActiveMember   10000 non-null  category
 11  EstimatedSalary  10000 non-null  float64 
 12  Exited           10000 non-null  int64   
 13  Status           10000 non-null  object  
dtypes: category(4), float64(2), int64(6), object(2)
memory usage: 820.9+ KB


RowNumber — номер строки таблицы (это лишняя информация, поэтому можете сразу от неё избавиться)
CustomerId — идентификатор клиента
Surname — фамилия клиента
CreditScore — кредитный рейтинг клиента (чем он выше, тем больше клиент брал кредитов и возвращал их)
Geography — страна клиента (банк международный)
Gender — пол клиента
Age — возраст клиента
Tenure — сколько лет клиент пользуется услугами банка
Balance — баланс на счетах клиента в банке
NumOfProducts — количество услуг банка, которые приобрёл клиент
HasCrCard — есть ли у клиента кредитная карта (1 — да, 0 — нет)
IsActiveMember — есть ли у клиента статус активного клиента банка (1 — да, 0 — нет)
EstimatedSalary — предполагаемая заработная плата клиента
Exited — статус лояльности (1 — ушедший клиент, 0 — лояльный клиент)

# 9.1. Каково соотношение ушедших и лояльных клиентов? 

Покажите это на графике и дайте комментарий по соотношению.

In [3]:
exited_cus = churn_data.groupby('Status', as_index=False)['CustomerId'].count()
display(exited_cus)
fig = px.pie(exited_cus, 
            values='CustomerId', 
            names='Status',
            hole=0.65,
            width= 600,
            height= 600,
            
            )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="top",
    y=0.0,
    xanchor="center",
    x=0.5,
    font_size=22),
    showlegend=False,
    
    annotations=[dict(text='Соотношение<br> ушедших и лояльных <br>клиентов', x=0.5, y=0.5, font_size=24, showarrow=False)])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')

fig.show()
fig.write_html("data/plots/1_pie_loyalty_percent.html")

,Status,CustomerId
0,Лояльные клиенты,7963
1,Ушедшие клиенты,2037


<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/1_pie_loyalty_percent.png?raw=true alt="drawing";"></center> <br>
Как мы видим отток клиентов хоть и существует не является критичным, дальнейшие изыскания покажут в каком направлении работать для удержания клиентов.

# 9.2. Распределение остатков на счетах клиентов, у которых на счету больше 2 500 долларов. 
Описание распределения и выводы.

In [4]:
churn_data.sort_values(by='Status', ascending= True, inplace= True)
mask_balance = (churn_data['Balance'] > 2500) #& (churn_data['Balance'] < 200000)
fig = px.histogram(churn_data[mask_balance], x="Balance",  marginal="box", nbins=30, width=1000 , height= 500)
fig.update_layout(title=dict(text= 'Распределение остатков на счетах у клиентов', yanchor="top", x= 0.5, font_size = 20),
    legend=dict(
    # orientation="h",
    # yanchor="top",
    # y=-0.1,
    # xanchor="center",
    # x=0.5, 
    font_size=18),
    showlegend=True
    
    )

fig.show()
fig.write_html("data/plots/2_hist_balance_distibution.html")


<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/2_hist_balance_distibution.png?raw=true alt="drawing";"></center> <br>
Из данного распределения мы видим, что нашей целевой группой являтся клиенты с остатками баланса на счетах в диапазоне от 40 тыс долларов до 200 тысяч долларов,<Br> ядро целевой группы - 2 & 3 квартиль от 100 до 140 тысяч долларов, медиана 120 тыс долларов.

# 9.3. Распределение баланса клиентов в разрезе признака оттока. 
Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? 
Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [5]:
mask_balance = churn_data['Balance'] > 2500
fig = px.histogram(churn_data[mask_balance], x="Balance", color='Status',  marginal="box", nbins=30, width=1150 , height= 500)
fig.update_layout(title=dict(text= 'Распределение остатков на счетах у клиентов<Br>в разрезе признака оттока', yanchor="top", x= 0.5, font_size = 20),
    legend=dict(
    # orientation="h",
    # yanchor="top",
    # y=-0.1,
    # xanchor="center",
    # x=0.5, 
    font_size=18),
    showlegend=True
    
    )

fig.show()
fig.write_html("data/plots/3_hist_balans_loyalty.html")
<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/2_hist_balance_distibution.png?raw=true alt="drawing";"></center> <br>

SyntaxError: invalid syntax (825820334.py, line 17)

<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/3_hist_balans_loyalty.png?raw=true alt="drawing";"></center> <br>
Наши данные показывают, что теряем клиентов в основном из ядра нашей целевой аудитории. <Br>

# 9.4. Pаспределение возраста в разрезе признака оттока. 
В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [ ]:
# mask_balance = churn_data['Balance'] > 2500 [mask_balance]
fig1 = px.histogram(churn_data,
                   x="Age", color='Status',  marginal="box", nbins=40, width=1150 , height= 500)
fig1.update_layout(title=dict(text= 'Распределение возраста клиентов<Br>в разрезе признака оттока', yanchor="top", x= 0.5, font_size = 20),
    legend=dict(
    # orientation="h",
    # yanchor="top",
    # y=-0.1,
    # xanchor="center",
    # x=0.5, 
    font_size=18),
    showlegend=True
    
    )

# подготовка данных к измененнию возраста клиентов старше 60 лет
no_need = ['IndicatorCode', 'IndicatorName', 'VariableName', 'MeasurementName', 'CountryCode', 'Alpha3Code', 'Unnamed: 9']

baby_woman = pd.read_csv('data/data_woman_baby.csv', encoding= 'latin-1')
baby_woman = baby_woman.drop(no_need, axis=1)
baby_woman['Value'] = baby_woman['Value'].apply(lambda x: x if x != '..' else np.nan).astype('float')
baby_woman.dropna(axis= 0, inplace = True)
first_baby_age = baby_woman.groupby('CountryName', as_index=False)['Value'].mean()
# display(first_baby_age.info())

woman_mar = pd.read_csv('data/data_woman_mar.csv', encoding= 'latin-1')
woman_mar = woman_mar.drop(no_need, axis=1)
woman_mar['Value'] = woman_mar['Value'].apply(lambda x: x if x != '..' else np.nan).astype('float')
woman_mar.dropna(axis= 0, inplace = True)
first_mar_woman = woman_mar.groupby('CountryName', as_index=False)['Value'].mean()
# display(baby_woman.info())

man_mar = pd.read_csv('data/data_man_mar.csv', encoding= 'latin-1')
man_mar = man_mar.drop(no_need, axis=1)
man_mar['Value'] = man_mar['Value'].apply(lambda x: x if x != '..' else np.nan).astype('float')
man_mar.dropna(axis= 0, inplace = True)
first_mar_man = man_mar.groupby('CountryName', as_index=False)['Value'].mean()
# display(man_mar.describe())


def new_age_over_60(df):
    for line in range(len(df)):
        if (df.loc[line, 'Gender'] == 'Female') and (df.loc[line, 'Age'] >= 60):
            df.loc[line, 'NewAge'] = round(df.loc[line, 'Age'] - df.loc[line, 'Value_x'])
        elif (df.loc[line, 'Gender'] == 'Male') and (df.loc[line, 'Age'] >= 60):
            df.loc[line, 'NewAge'] = round((df.loc[line, 'Age'] - df.loc[line, 'Value_x']) -(df.loc[line, 'Value'] - df.loc[line, 'Value_y']))
        else:
            df.loc[line, 'NewAge'] = df.loc[line, 'Age']
    df.drop(labels=['Value', 'Value_x', 'Value_y'],  axis= 1, inplace = True)
    return(df)



new_churn = churn_data.merge(first_baby_age, how='left', left_on='Geography', right_on='CountryName')
new_churn = new_churn.merge(first_mar_woman, how='left', left_on='Geography', right_on='CountryName')
new_churn = new_churn.merge(first_mar_man, how='left', left_on='Geography', right_on='CountryName')

no_need2 = ['CountryName_x','CountryName_y', 'CountryName']
new_churn.drop(labels= no_need2, axis= 1, inplace= True)

new_churn = new_age_over_60(new_churn)
# display(new_churn[new_churn['Age'] >= 60]) #[mask_balance],

new_churn.sort_values(by='Status', ascending= True, inplace= True)
fig2 = px.histogram(new_churn,
                   x="NewAge", color='Status',  marginal="box", nbins=30, width=1150 , height= 500)
fig2.update_layout(title=dict(text= 'Распределение возраста клиентов<Br>в разрезе признака оттока (очищено)', yanchor="top", x= 0.5, font_size = 20),
    legend=dict(
    # orientation="h",
    # yanchor="top",
    # y=-0.1,
    # xanchor="center",
    # x=0.5, 
    font_size=18),
    showlegend=True
    
    )

display(fig1.show())
fig.write_html("data/plots/4_hist_age_loyalty_base.html")

fig2.show()
fig.write_html("data/plots/4_hist_age_loyalt_clear.html")


<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/4_hist_age_loyalty_base.png?raw=true alt="drawing";"></center> <br>

Данные говорят о том, что нас покидают клиенты в основном с 38 до 51 года (2&3 квартиль), ядро нашей целевой аудитории с 31 до 41 года. <Br>
Больше всего выбрасов в группу лояльных клиентов в части людей старше 60 лет, что объясняется консервативностью данной возрастной ауидории и скорее всего тем, что финансовые вопросы данной категории решают за них их дети которые и выбирают наш банк. (вполне возможно добавить их в группы с корректировкой по возрасту, для этого сделаем обработку скачаем с https://w3.unece.org/ информацию в разрезе стран по среднему возрасту женщины, когда она рождает первого ребенка, среднему возрасту женщины и мужчины когда они вступают в первый брак соответственно у женщин старше 60 лет мы при определении группы вычитаем средний возраст рождения первого ребенка, у мужчины дополнительно вычитаем разницу в возрасте  вступления в брак мужчины и женщины) <Br> (Пока вроде смысла эта работа не имела)

<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/4_hist_age_loyalty_clear.png?raw=true alt="drawing";"></center> <br>

# 9.5. Взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. 
Добавьте расцветку по признаку оттока клиентов. <Br>Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [ ]:


fig = px.scatter(churn_data, x="EstimatedSalary", y="CreditScore", color="Status", 
                 marginal_x="box", marginal_y="box", width=1300, height= 1300,
                  title="Взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты")
fig.show()
fig.write_html("data/plots/5_scater_creditrating_salary.html")


<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/5_scater_creditrating_salary.png?raw=true alt="drawing";"></center> <br>

Рейтинг наших лояльных клиентов в общей массе выше тех, что ушли от нас, на грани статистической погрешности. <Br>Радует одно что клиенты с кредитным рейтингом ниже 400 уходят от нас (19 человек), но с другой стороны это довольно состоятельные клиенты c медианным балансом 111.1 тыс долларов и медианной зарплатой 137.7 тыс долларов. <Br>В остальном какой либо закономерности не замечено (так же при использовании логорифмических шкал)


# 9.6. Соотношение ушедших клиентов к лояльным, в разрезе мужчин и женщин.

In [ ]:
churn_data.sort_values(by='Gender', ascending=False, inplace=True)
fig = px.histogram(churn_data, x="Gender", color="Gender", pattern_shape="Status" , width=1100, 
                   
                #    text_auto=True
                   )

female_ex= str(round(churn_data[churn_data['Gender'] == 'Female']['Exited'].mean()*100, 1)) + '%'
male_ex= str(round(churn_data[churn_data['Gender'] == 'Male']['Exited'].mean()*100, 1)) + '%'
percent_ex=['', male_ex,'', female_ex]
# display(fig.data)

for i in range(4):
   fig.data[i].text = percent_ex[i]

fig.update_traces(textposition='inside', 
                  textfont_size = 18
                  )
fig.show()
fig.write_html("data/plots/6_bar_gender_status.html")

<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/6_bar_gender_status.png?raw=true alt="drawing";"></center> <br>
В части женщин наблюдается значительно больший отток, чем среди мужчин.

# 9.7. Как отток клиентов зависит от числа приобретённых у банка услуг? Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [ ]:
df = churn_data
df.head()

x,y = 'NumOfProducts', 'Status'

df1 = df.groupby(x)[y].value_counts(normalize=True)
df1 = df1.mul(100)
df1 = df1.rename('percent').reset_index()

g = sns.catplot(x=x,y='percent',hue=y,kind='bar',data=df1)
g.ax.set_ylim(0,100)
g.title = 'Соотношение лояльных и ушевших клиетов в разрезе количества продуктов'
for p in g.ax.patches:
    txt = str(p.get_height().round(2)) + '%'
    txt_x = p.get_x() 
    txt_y = p.get_height()
    g.ax.text(txt_x,txt_y,txt);

<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/7_bar_status_numofproducts.png?raw=true alt="drawing";"></center> <br>
Исходя из данного графика необходимо провести дополнительный анализ продуктов и их влияние на наших клиентов. Ушедших клиентов с 3 & 4 продуктами значительно больше, видимо последний опыт взаимодействия с банком был "очень не очень". 

# 9.8. Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. 
Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [ ]:

churn_data['ActiveStatus'] = churn_data['IsActiveMember'].apply(lambda x: 'Активные клиенты' if x == 0 else 'Потенциал роста')
# display(churn_data.head())
churn_data.sort_values(by=['ActiveStatus', 'Exited'], ascending=[False, True], inplace=True)
fig = px.histogram(churn_data, x="ActiveStatus", color="ActiveStatus", pattern_shape=churn_data['Status'] , width=1100, 
                   
                #    text_auto=True
                   )

active_ex= str(round(churn_data[churn_data['ActiveStatus'] == 'Активные клиенты']['Exited'].mean()*100, 1)) + '%'
pasive_ex= str(round(churn_data[churn_data['ActiveStatus'] == 'Потенциал роста']['Exited'].mean()*100, 1)) + '%'
percent_ex=['', pasive_ex,'', active_ex]
# display(fig.data)

for i in range(4):
   fig.data[i].text = percent_ex[i]

fig.update_traces(textposition='inside', 
                  textfont_size = 18
                  )
fig.show()
fig.write_html("data/plots/8_bar_active_loyalty.html")

<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/8_bar_active_loyalty.png?raw=true alt="drawing";"></center> <br>
Среди активных клиентов ушедших значительно больше, чем среди клиентов с потенциалом роста, необходимо провести опрос на тему каких продуктов им не хватает в нашем банке, или какие продукты и в чем их не устраивают.

# 9.9. Картограмма зависимости процента ушедших клиентов от страны.

In [ ]:


choropleth_data= churn_data.groupby('Geography', as_index= False )['Exited'].mean()
choropleth_data['Exited'] = choropleth_data['Exited']*100
#строим график
fig = px.choropleth(
    data_frame=choropleth_data, #DataFrame
    locations="Geography", #столбец с локациями
    locationmode = "country names", #режим сопоставления локаций с базой Plotly
    color='Exited', #от чего зависит цвет
    # animation_frame="date", #анимационный бегунок
    range_color=[10, 35], #диапазон цвета
    title='Процент ушедших клиентов в разрезе стран', #заголовок
    width=800, #ширина
    height=400, #высота
    center = {"lat": 45.3220, "lon": 5.0415},
    color_continuous_scale='Reds' #палитра цветов
)
fig.update_layout(title_x=0.5)
fig.update_layout(margin={"r":10,"t":40,"l":10,"b":10}, autosize = True)
fig.update_geos(projection_scale=9)

# 47.3220° N, 5.0415° E Dijon

fig.show();
fig.write_html("data/plots/9_chronopleth_country_loyalty.html")

<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/9_chronopleth_country_loyalty.png?raw=true alt="drawing";"></center> <br>
В соответствии с предстваленными данными что то происходит в Германии 32% клиентов ушли за рассмотриваемый период из банка что в 2 раза больше чем во Франции или Испании за то же время.

# 9.10 Зависимость лояльныих и ушедших клиентов от категориального признака кредитный рейтиг.
Переведите числовой признак CreditScore в категориальный. Для этого воспользуйтесь функцией get_credit_score_cat(), которая приведена ниже. Примените её к столбцу CreditScore и создайте новый признак CreditScoreCat — категории кредитного рейтинга.

In [ ]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"
    
churn_data['CreditScoreCat'] = churn_data['CreditScore'].apply(get_credit_score_cat)
# display(churn_data.head())

churn_data=churn_data.sort_values(by=['CreditScore', 'Exited'], ascending=[True, False])

pivot = churn_data.pivot_table(aggfunc='mean',
                                values='Exited',
                                columns='Tenure',
                                index='CreditScoreCat'
                                )
# pivot.style.format('{:.1f}')
# display(pivot)

heatmap = sns.heatmap(data=pivot, cmap='YlGnBu')
heatmap.margins(x=0.05, y=0.01)
heatmap.set_title('Зависимость % ушедших клиентов от категориального кредитного рейтинга и длительности сотрудничества с ним', fontsize=16);
fig.write_html("data/plots/10_heatmap_loaylty_rating_years.html")

<center> <img src = https://github.com/Denis-python/VS_code/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F%20Skillfactory/PY13/Data/plots_png/10_heatmap_loaylty_rating_years.png?raw=true alt="drawing";"></center> <br>
Клиенты с более низким рейтингом склонны уходить чаще из банка, программа лояльности направленная на изменение привычек клиента влияющих на рейтиг через экономические агенты может существенно улучшить показатели удержания клиентов и, в то же время,  улучшая качество портфеля продуктов банка.
На втором месте по проценту ушедших клиентов стоят клиенты с кредитным рейтингом Excellent, они осознают что обладая достаточно высоким кредитным рейтингом они могут получать лучшие предложения от любого банка, соответственно им необходимо предлагать более выгодные условия, чем предлагают конкуренты.